Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.


```
SELECT name 
FROM Facilities
WHERE membercost > 0;
```

Q2: How many facilities do not charge a fee to members? 

```
SELECT COUNT(name) 
FROM Facilities
WHERE membercost = 0
```

Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

```
SELECT facid, name, membercost, monthlymaintenace
FROM Facilities
WHERE membercost < monthlymaintenance * 20 / 100
```

Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.

```
SELECT *
FROM Facilities
WHERE facid IN (1, 5)
```

Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.

```
SELECT 
    name, 
    monthlymaintenance,
    (CASE WHEN monthlymaintenance <= '100' THEN 'cheap'
          ELSE 'expensive' END) AS price_label
FROM Facilities
```

Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. 

```
SELECT firstname, surname
FROM Members
WHERE joindate = 
    (SELECT MAX(joindate)
     FROM Members)
```

Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

```
SELECT
	f.name as facility_name,
	CONCAT(m.firstname, ' ', m.surname) AS fullname
FROM Members AS m
INNER JOIN Bookings AS b
ON m.memid = b.memid
INNER JOIN Facilities AS f
ON f.facid = b.facid
GROUP BY fullname
ORDER BY fullname
```

Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

```
SELECT 
	f.name AS facility_name, 
	CONCAT( m.firstname, ' ', m.surname ) AS fullname, 
	(CASE WHEN b.memid =0
	THEN f.guestcost * b.slots
	ELSE f.membercost * b.slots END) AS cost
FROM Members AS m
INNER JOIN Bookings AS b 
ON m.memid = b.memid
INNER JOIN Facilities AS f 
ON f.facid = b.facid
WHERE b.starttime LIKE '2012-09-14%'
AND ((b.memid = 0 AND f.guestcost * b.slots > 30)
	OR (b.memid <> 0 AND f.membercost * b.slots > 30))
ORDER BY cost DESC
```

Q9: This time, produce the same result as in Q8, but using a subquery.

```
SELECT
	facility,
	fullname,
	cost
FROM (
    SELECT 
    	f.name AS facility,
    	CONCAT(m.firstname, ' ', m.surname) AS fullname,
		(CASE WHEN b.memid = 0 THEN b.slots * f.guestcost
         ELSE b.slots * f.membercost END) AS cost,
    	b.starttime
    FROM Members AS m
    INNER JOIN Bookings as b
    ON m.memid = b.memid
    INNER JOIN Facilities as f
    ON f.facid = b.facid
) AS new
WHERE new.starttime LIKE '2012-09-14%' AND cost > 30
ORDER BY cost DESC
```

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [1]:
from sqlalchemy import create_engine
import pandas as pd
database = 'sqlite:///sqlite_db_pythonsqlite.db'
engine = create_engine(database)

In [2]:
con = engine.connect()
querry = """
SELECT
	facility,
	revenue
FROM (
    SELECT
    	f.name AS facility,
    	SUM(CASE WHEN b.memid = 0 THEN b.slots * f.guestcost
         ELSE b.slots * membercost END) AS revenue
    FROM Members AS m
    INNER JOIN Bookings AS b
    ON m.memid = b.memid
    INNER JOIN Facilities AS f
    ON b.facid = f.facid
    GROUP BY facility
    ) AS newtab
WHERE revenue < 1000
ORDER BY revenue
"""
rs = con.execute(querry)
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()
df

,facility,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [3]:
con = engine.connect()
querry = """
SELECT 
	(m1.surname || m1.firstname) AS Member_name,
	(CASE WHEN m1.recommendedby < 1 THEN 'No recommender'
    ELSE (m2.surname || m2.firstname) END) AS Recommender
FROM Members AS m1
INNER JOIN Members AS m2
ON m1.recommendedby = m2.memid
ORDER BY Member_name
"""
rs = con.execute(querry)
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()
df

,Member_name,Recommender
0,BaderFlorence,StibbonsPonder
1,BakerAnne,StibbonsPonder
2,BakerTimothy,FarrellJemima
3,BootheTim,RownamTim
4,ButtersGerald,SmithDarren
5,CoplinJoan,BakerTimothy
6,CrumpetErica,SmithTracy
7,DareNancy,JopletteJanice
8,GentingMatthew,ButtersGerald
9,HuntJohn,PurviewMillicent


Q12: Find the facilities with their usage by member, but not guests

In [4]:
con = engine.connect()
querry = """
SELECT 
	f.name AS facility,
    COUNT(b.memid) AS member_use_time
FROM Facilities AS f
INNER JOIN Bookings AS b
ON f.facid = b.facid
WHERE b.memid > 0
GROUP BY facility
ORDER BY member_use_time
"""
rs = con.execute(querry)
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()
df

,facility,member_use_time
0,Massage Room 2,27
1,Squash Court,195
2,Tennis Court 2,276
3,Tennis Court 1,308
4,Badminton Court,344
5,Table Tennis,385
6,Massage Room 1,421
7,Snooker Table,421
8,Pool Table,783


Q13: Find the facilities usage by month, but not guests


In [5]:
con = engine.connect()
querry = """
SELECT 
	facility,
    member_use_time,
	month
FROM (
    SELECT 
    	f.name AS facility,
    	COUNT(memid) AS member_use_time,
    	strftime('%m', starttime) AS month
    FROM Facilities AS f
	INNER JOIN Bookings AS b
	ON f.facid = b.facid
	WHERE b.memid > 0
	GROUP BY facility, month) AS newtab

ORDER BY facility
"""
rs = con.execute(querry)
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()
df

,facility,member_use_time,month
0,Badminton Court,51,07
1,Badminton Court,132,08
2,Badminton Court,161,09
3,Massage Room 1,77,07
4,Massage Room 1,153,08
5,Massage Room 1,191,09
6,Massage Room 2,4,07
7,Massage Room 2,9,08
8,Massage Room 2,14,09
9,Pool Table,103,07
